In [1]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')

    from google.colab import drive
    drive.mount('/content/drive')   

    import os
    os.chdir('/content/drive/My Drive/Deep_learning_project/')

else:
    print('Not running on CoLab')

Not running on CoLab


In [10]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.distributions import Normal
from torch.utils.data import TensorDataset, DataLoader

import datetime
import itertools
from tqdm import tqdm
import pickle
from torch.optim import lr_scheduler

from sklearn.model_selection import train_test_split

from modules import *

In [11]:
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")

In [12]:
df = pd.read_csv('GM_preparedData.csv')

In [13]:
# Define rectangle where "valid data belongs":

upper_right = (55.880120, 12.707644)
lower_left = (55.548626, 12.061736)

# Clean data
df_clean = df.loc[(df['startPositionLat'] >= lower_left[0]) & (df['startPositionLat']<=upper_right[0])]
df_clean = df_clean.loc[(df_clean['startPositionLng'] >= lower_left[1]) & (df_clean['startPositionLng']<=upper_right[1])]

df_clean = df_clean.loc[(df_clean['endPositionLat'] >= lower_left[0]) & (df_clean['endPositionLat']<=upper_right[0])]
df_clean = df_clean.loc[(df_clean['endPositionLng'] >= lower_left[1]) & (df_clean['endPositionLng']<=upper_right[1])]

df_clean = df_clean.dropna(
    axis=0, 
    subset=['startPositionLat', 'startPositionLng', 'endPositionLat', 'endPositionLng']
)

In [14]:
data_columns = ['startPositionLng', 'startPositionLat', 'endPositionLng', 'endPositionLat']
data = df_clean[data_columns]

In [15]:
train, test = train_test_split(data.values, test_size=0.1, random_state=42)
df_train = pd.DataFrame(train, columns=data.columns)
df_test = pd.DataFrame(test, columns=data.columns)

#Normalize training data
mu_lat_start = df_train['startPositionLat'].mean()
sig_lat_start = df_train['startPositionLat'].std()

mu_lng_start = df_train['startPositionLng'].mean()
sig_lng_start = df_train['startPositionLng'].std()

mu_lat_end = df_train['endPositionLat'].mean()
sig_lat_end = df_train['endPositionLat'].std()

mu_lng_end = df_train['endPositionLng'].mean()
sig_lng_end = df_train['endPositionLng'].std()

df_train['NormLngStart'] = df_train['startPositionLng'].apply(lambda x: (x-mu_lng_start)/sig_lng_start)
df_train['NormLatStart'] = df_train['startPositionLat'].apply(lambda x: (x-mu_lat_start)/sig_lat_start)

df_train['NormLngEnd'] = df_train['endPositionLng'].apply(lambda x: (x-mu_lng_end)/sig_lng_end)
df_train['NormLatEnd'] = df_train['endPositionLat'].apply(lambda x: (x-mu_lat_end)/sig_lat_end)

# Normalize validation set
test_mu_lat_start = df_test['startPositionLat'].mean()
test_sig_lat_start = df_test['startPositionLat'].std()

test_mu_lng_start = df_test['startPositionLng'].mean()
test_sig_lng_start = df_test['startPositionLng'].std()

test_mu_lat_end = df_test['endPositionLat'].mean()
test_sig_lat_end = df_test['endPositionLat'].std()

test_mu_lng_end = df_test['endPositionLng'].mean()
test_sig_lng_end = df_test['endPositionLng'].std()

df_test['NormLngStart'] = df_test['startPositionLng'].apply(lambda x: (x-test_mu_lng_start)/test_sig_lng_start)
df_test['NormLatStart'] = df_test['startPositionLat'].apply(lambda x: (x-test_mu_lat_start)/test_sig_lat_start)

df_test['NormLngEnd'] = df_test['endPositionLng'].apply(lambda x: (x-test_mu_lng_end)/test_sig_lng_end)
df_test['NormLatEnd'] = df_test['endPositionLat'].apply(lambda x: (x-test_mu_lat_end)/test_sig_lat_end)

In [16]:
data_train = df_train[['NormLngStart', 'NormLatStart', 'NormLngEnd', 'NormLatEnd']]
data_test = df_test[['NormLngStart', 'NormLatStart', 'NormLngEnd', 'NormLatEnd']]

# MODEL

In [17]:
mu = torch.zeros(4).to(device)
sigma = torch.ones(4).to(device)
base = Normal(mu, sigma)

net = RealNVP(in_features=4, prior=base, hidden_features=256, depth=6).to(device)
optimizer = torch.optim.Adam([p for p in net.parameters() if p.requires_grad], lr=1e-4)

pyro_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=25, verbose=True)

# Arbitrary clipping value to suppress exploding gradients
clipping_value = 1

In [18]:
X_train = torch.Tensor(data_train.values).to(device)
X_test = torch.Tensor(data_test.values).to(device)

train_dataset = TensorDataset(X_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [19]:
train_losses = []
validation_losses = []
save_flag = True
epochs = 500

try:
    for epoch in tqdm(range(epochs)):
        loss_epoch = 0

        # VALIDATION
        net.eval()

        validation_loss = -torch.mean(net.log_likelihood(X_test))
        validation_losses.append(validation_loss.item())
        
        pyro_scheduler.step(validation_loss)

        # TRAIN LOOP
        net.train()
        for batch_num, (X_batch) in enumerate(train_loader):
            X_batch = X_batch[0]
            
            optimizer.zero_grad()
            
            loss = -torch.mean(net.log_likelihood(X_batch))
            
            loss.backward()
            optimizer.step()

            loss_epoch += loss.item()

        loss_epoch = loss_epoch / (batch_num+1)
        train_losses.append(loss_epoch)

        if (epoch) % 1 == 0:
            print(f'Validation loss: {validation_loss}')
            print(f'Train loss: {loss_epoch}')

        if (epoch) % 50 == 0:
            torch.save(net.state_dict(), 'model_state_fix_prior_pickup_cond_updated_epoch_%d.pth' % (epoch+1))
    
    if save_flag:
        loss_dict = {'validation': validation_losses, 'train': train_losses}
        with open('loss_dict_pickup_cond_fix_prior.pkl', 'wb') as f:
            pickle.dump(loss_dict, f)

        torch.save(net.state_dict(), 'model_state_pickup_fix_prior_cond.pth')

except KeyboardInterrupt:
    if save_flag:
        loss_dict = {'validation': validation_losses, 'train': train_losses}
        with open('loss_dict_pickip_cond_fix_prior.pkl', 'wb') as f:
            pickle.dump(loss_dict, f)

        torch.save(net.state_dict(), 'model_state_fix_prior_pickup_cond.pth')

  0%|          | 1/500 [00:21<2:58:54, 21.51s/it]

Validation loss: 1.2166879177093506
Train loss: -0.5792530259079128


  0%|          | 1/500 [00:27<3:45:28, 27.11s/it]
